In [1]:
# Importing packages
from selenium import webdriver
import pandas as pd
import numpy as np
import math
from tqdm import tqdm_notebook as tqdm
import time 
import re

In [ ]:
#Script to check whether the institution is mentioned in the obituary and extract date of birth and date of death
# Choose University ID
#List of Instititions:
#Boston University 24
#Emory 66
#John Hopkins 102
#Indiana U 96
#Michigan State 132
#NYU 146
#NWU 153
#OSU 156
#Oregon State 161 
#Penn State 163 
#Princeton
#Purdue 
#Rutgers 177
#Stony Brook
#U Arizona
#UT Austin 229 
# UCSD 256 
#U of Cincinnati
#U of Colorado, Boulder 263
#UIUC 280 
#U Iowa 281
#U Kansas 282 
#U Michigan 
#U Oregon 325 
#U Penn 326 
#U Pittsburgh 327
#U Virginia 345
#UW Madison 347 
#WUSTL 358 

In [658]:
import mysql.connector

# Connect to the sql server
cnx = mysql.connector.connect(user='widr', password='nnamd28',
                              host='10.0.46.2',
                              database='rainer')

# Get attribute of cursor
mycursor = cnx.cursor()

# Select the table
mycursor.execute("SELECT * FROM umetrics_faculty_last_entry_2009_2016")

# Extract the names of the columns from the sql database
sequence = mycursor.column_names

# Create a list with all the information
myresult = mycursor.fetchall()


In [659]:
# Construct dataframe using name_columns
all_prof = pd.DataFrame(myresult, columns=sequence)
all_prof.head()
for col in all_prof.columns: 
    print(col)

umetrics_faculty_row_id
aa_row_id
Year
InstitutionId
InstitutionName
PersonId
InstitutionName.x
UnitId
UnitName
FacultyTitle
PersonName
FirstName
Nickname
MidName
LastName
Suffix
FacultyTypeId
RankTypeId
TitleTypeId
TaxonomyLevel01Id
TaxonomyLevel01Name
IPEDSId
StateAbbr
IsPublic
IsAAU
Gender
DegreeYear
DegreeTypeName
DegreeInstitutionID
DegreeInstitutionName


In [660]:
# Check ducplicates
all_prof.duplicated(['PersonName'])

# Rename columns which contain person´s name and university´s name and IDs
all_prof = all_prof.rename(columns={"PersonName": "name","PersonId": "person_id" , \
                                    "Year": "year", "InstitutionName.x": "university", "InstitutionId": "univ_id",\
                                   "FirstName": "firstname","LastName": "lastname"})


In [661]:
for index, row in all_prof.head().iterrows():
     # access data using column names
     print(index, row['firstname'], row['Nickname'], row['lastname'], row['university'])

0 DAVID NULL CRIBB Air Force Institute of Technology
1 RYAN NULL THOMAS Air Force Institute of Technology
2 RANDALL NULL PASCHALL Air Force Institute of Technology
3 CHRISTOPHER NULL WEST Air Force Institute of Technology
4 STEVEN NULL THORSEN Air Force Institute of Technology


In [662]:
# Sort from the latest year and name 
# This is important since we take only the last appereance of the professor in the University (but you have done it in sql before)
# Then afet when we get the date from genealogy we can compare how many years have passed until his death
all_prof = all_prof.sort_values(by=['year', 'name'], ascending=False)

# Eliminate the duplicates by preserving the first observation 
all_prof = all_prof.drop_duplicates(subset=['name', 'university', 'year'], keep = 'first')


In [663]:
# WE DONT HAVE INFORMATION OF YEAR OF BIRTH NIETHER AGE 

# now we will keep just information before 2014 since in the genealogy webpage we have no information after that period
# This was done from you before

#  This is the line we need to change to include all  data 
# all_prof = all_prof[all_prof["year"] <= 2014]

# resort for looking first from 2009-2014 
all_prof = all_prof.sort_values(by=['year', 'name'], ascending=True)
all_prof["year"].value_counts()
all_prof["univ_id"].value_counts()
all_prof

,umetrics_faculty_row_id,aa_row_id,year,univ_id,InstitutionName,person_id,university,UnitId,UnitName,FacultyTitle,...,TaxonomyLevel01Name,IPEDSId,StateAbbr,IsPublic,IsAAU,Gender,DegreeYear,DegreeTypeName,DegreeInstitutionID,DegreeInstitutionName
17342,37497,129598,2009,251,147875,203116,"University of California, Berkeley",1708,"Business, School of",E.T. Grether Professor,...,Business Administration,110635,CA,1,1,M,NA,NA,NA,NA
19460,47888,139989,2009,258,97,129,"University of California, Santa Barbara",4682,"English, Department of",None,...,English Language and Literature,110705,CA,1,1,U,NA,NA,NA,NA
24970,58454,161931,2009,281,52,64,"University of Iowa, The",2356,"Molecular Physiology and Biophysics, Departmen...",Professor,...,Biophysics,153658,IA,1,1,M,1955,"Bachelor of Medicine, Bachelor of Surgery",555736,"Ains Chams University, Cairo"
8425,22014,85249,2009,163,141411,189867,"Pennsylvania State University, The",693,"Dairy and Animal Science, Department of",Associate Professor,...,Animal Sciences,214777,PA,1,1,M,1991,Doctor of Philosophy,277,University of Houston
5177,11497,63953,2009,132,179139,273436,Michigan State University,6343,"Linguistics and Germanic, Slavic, Asian and Af...",AST PROFESSOR,...,"Languages, various",171100,MI,1,1,M,NA,NA,NA,NA
17753,38861,130962,2009,252,148420,204166,"University of California, Davis",1757,"Plant Sciences, Department of",Associate Professor,...,Plant Sciences,110644,CA,1,1,M,NA,NA,NA,NA
28889,61733,181114,2009,304,154095,217863,University of Missouri,9936,"Learning, Teaching, and Curriculum, Department of","PROF, AST RESRCH",...,Curriculum and Instruction,178396,MO,1,1,F,NA,NA,NA,NA
33053,72736,224120,2009,354,425,555,Virginia Polytechnic Institute and State Unive...,10060,"Foreign Languages, Department of",Assistant Professor,...,"Languages, various",233921,VA,1,0,M,NA,NA,NA,NA
25596,59959,163436,2009,281,155,200,"University of Iowa, The",2337,"Electrical and Computer Engineering, Departmen...",Assistant Professor,...,Electrical Engineering,153658,IA,1,1,M,2001,Doctor of Philosophy,538758,Utrecht University
7277,16103,76333,2009,153,524,677,Northwestern University,9027,"Music Studies, Department of",Assoc Professor,...,"Music, General",147767,IL,0,1,M,2003,Doctor of Philosophy,156,"Ohio State University, The"


In [664]:
# Create range for date of death 
#all_prof.rename(columns={'year' : 'death_1'}, inplace=True)

all_prof['death_1'] = '01/01/2014'
all_prof['death_2'] = '06/30/2019'


for index, row in all_prof.head().iterrows():
     # access data using column names
     print(index, row['name'], row['death_1'], row['death_2'])

17342 AAKER, DAVID A 01/01/2014 06/30/2019
19460 ABBOTT, H PORTER 01/01/2014 06/30/2019
24970 ABBOUD, FRANCOIS M 01/01/2014 06/30/2019
8425 ABCOUWER, STEVEN 01/01/2014 06/30/2019
5177 ABDALLA, MAHMOUD IBRAHIM 01/01/2014 06/30/2019


In [665]:
# We need to change the type of input 

all_prof['death_1'] = all_prof['death_1'].astype('str')
all_prof['death_2'] = all_prof['death_2'].astype('str')

all_prof['death_1'].head()


17342    01/01/2014
19460    01/01/2014
24970    01/01/2014
8425     01/01/2014
5177     01/01/2014
Name: death_1, dtype: object

In [666]:
all_prof['title'] = "Professor"
all_prof['death_2'].head()
all_prof['name']

17342                AAKER, DAVID A
19460              ABBOTT, H PORTER
24970            ABBOUD, FRANCOIS M
8425               ABCOUWER, STEVEN
5177       ABDALLA, MAHMOUD IBRAHIM
17753                 ABEL, STEFFEN
28889               ABELL, SANDRA K
33053              ABIRAGI, ANTHONY
25596             ABRAMOFF, MICHAEL
7277                ABRIL, CARLOS R
25074              ABROMOWITZ, PAUL
24975              ACCARDI, ALESSIO
19052             ACEVEDO, RAFAEL D
7928              ACHTERMAN, GAIL L
24962                ADAMS, BRIAN D
24901          ADAMS, CHRISTOPHER M
7941                  ADAMS, DARIUS
33263                  ADAMS, JAMIE
33250                 ADAMS, LAUREN
8473                    ADAMS, TODD
8041           ADAMS, WESLEY THOMAS
30826        ADAMSON, PETER CHARLES
33124                 ADLER, JOHN A
2580               ADNERSSON, STEEN
24304    AGARWAL-TRONETTI, RAJSHREE
11672                AGBEH, ANTHONY
8376                     AGEE, MARK
12317              AGGARWAL,

In [667]:
#all_prof['university'].value_counts
#all_prof['university'].str.findall('Purdue')

all_prof[all_prof['university'].str.match('Ohio')]

,umetrics_faculty_row_id,aa_row_id,year,univ_id,InstitutionName,person_id,university,UnitId,UnitName,FacultyTitle,...,IsPublic,IsAAU,Gender,DegreeYear,DegreeTypeName,DegreeInstitutionID,DegreeInstitutionName,death_1,death_2,title
7658,18706,79312,2009,156,4365,5578,"Ohio State University, The",10479,"Hematology and Oncology, Division of",Professor,...,1,1,F,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor
7663,19029,79635,2009,156,134131,175233,"Ohio State University, The",10509,"Microbiology, Department of",Associate Professor,...,1,1,M,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor
7651,18374,78980,2009,156,102429,132898,"Ohio State University, The",10468,"Food, Agricultural and Biological Engineering,...",Professor,...,1,1,M,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor
7522,16873,77479,2009,156,103009,133705,"Ohio State University, The",10580,"Veterinary Clinical Sciences, Department of",Associate Professor,...,1,1,M,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor
7662,18976,79582,2009,156,8017,10284,"Ohio State University, The",15487,"Public Health, College of",Associate Professor,...,1,1,M,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor
7653,18502,79108,2009,156,7813,9991,"Ohio State University, The",10462,"Evolution, Ecology and Organismal Biology, Dep...",Professor,...,1,1,M,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor
7631,17467,78073,2009,156,227920,393503,"Ohio State University, The",10479,"Hematology and Oncology, Division of",Professor,...,1,1,M,1991,Doctor of Medicine,249,University of Arkansas for Medical Sciences,01/01/2014,06/30/2019,Professor
7645,17795,78401,2009,156,105504,136891,"Ohio State University, The",10580,"Veterinary Clinical Sciences, Department of",Professor,...,1,1,M,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor
7475,16652,77258,2009,156,193163,300969,"Ohio State University, The",10466,"Food Science and Technology, Department of",Professor,...,1,1,M,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor
7550,17029,77635,2009,156,14434,18709,"Ohio State University, The",10487,"Immunology and Rheumatology, Division of",Professor,...,1,1,U,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor


In [668]:
## Choose University ID ,  For instance  "Boston = 24" 

univ = 156
univ_prof = all_prof[all_prof['univ_id']== univ]
univ_prof


,umetrics_faculty_row_id,aa_row_id,year,univ_id,InstitutionName,person_id,university,UnitId,UnitName,FacultyTitle,...,IsPublic,IsAAU,Gender,DegreeYear,DegreeTypeName,DegreeInstitutionID,DegreeInstitutionName,death_1,death_2,title
7658,18706,79312,2009,156,4365,5578,"Ohio State University, The",10479,"Hematology and Oncology, Division of",Professor,...,1,1,F,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor
7663,19029,79635,2009,156,134131,175233,"Ohio State University, The",10509,"Microbiology, Department of",Associate Professor,...,1,1,M,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor
7651,18374,78980,2009,156,102429,132898,"Ohio State University, The",10468,"Food, Agricultural and Biological Engineering,...",Professor,...,1,1,M,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor
7522,16873,77479,2009,156,103009,133705,"Ohio State University, The",10580,"Veterinary Clinical Sciences, Department of",Associate Professor,...,1,1,M,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor
7662,18976,79582,2009,156,8017,10284,"Ohio State University, The",15487,"Public Health, College of",Associate Professor,...,1,1,M,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor
7653,18502,79108,2009,156,7813,9991,"Ohio State University, The",10462,"Evolution, Ecology and Organismal Biology, Dep...",Professor,...,1,1,M,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor
7631,17467,78073,2009,156,227920,393503,"Ohio State University, The",10479,"Hematology and Oncology, Division of",Professor,...,1,1,M,1991,Doctor of Medicine,249,University of Arkansas for Medical Sciences,01/01/2014,06/30/2019,Professor
7645,17795,78401,2009,156,105504,136891,"Ohio State University, The",10580,"Veterinary Clinical Sciences, Department of",Professor,...,1,1,M,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor
7475,16652,77258,2009,156,193163,300969,"Ohio State University, The",10466,"Food Science and Technology, Department of",Professor,...,1,1,M,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor
7550,17029,77635,2009,156,14434,18709,"Ohio State University, The",10487,"Immunology and Rheumatology, Division of",Professor,...,1,1,U,NA,NA,NA,NA,01/01/2014,06/30/2019,Professor


In [669]:
# all_prof = all_prof.sort_values(by=['death_1'], ascending=False)

# Create table to fill up with the information
columns_names = ['Name_web', 'Date_Death']

# scrap_data will contain information from the webpage
scrap_data_1 = pd.DataFrame(columns=columns_names)
scrap_data_1 = scrap_data_1.fillna(0)


# Create table to fill up with the information

# scrap_data will contain information from the webpage
scrap_data_2 = pd.DataFrame(columns=columns_names)
scrap_data_2 = scrap_data_2.fillna(0)


# Create text file where we save all information
f = open("test_1.txt","w+")
g = open("test_2.txt","w+")

# Code to find people in genealogybank
driver = webdriver.Chrome('C:/Users/Alexander/Documents/Chrome_Driver/chromedriver.exe')
driver.get('http://www.legacy.com/obituaries/legacy/obituary-search.aspx?daterange=30&countryid=0&stateid=all&affiliateid=all')

# Close advertisement
driver.find_element_by_xpath('//*[@id="qcCmpButtons"]/button').click()

# Maximize the window
driver.maximize_window()

 # Set the state of last residence
driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_uxSearchWideControl_ddlCountry"]/option[11]').click()


for index, row in tqdm(univ_prof.iterrows()):
    try:
        # Set the state of last residence
        driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_uxSearchWideControl_ddlCountry"]/option[11]').click()


        # range of death
        driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_uxSearchWideControl_ddlSearchRange"]/option[10]').click()

        death_begin = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_uxSearchWideControl_txtStartDate"]')
        death_begin.send_keys(row['death_1'])

        death_end = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_uxSearchWideControl_txtEndDate"]')    
        death_end.send_keys(row['death_2'])

        # type the Firstname 
        keyword = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_uxSearchWideControl_txtFirstName"]')
        keyword.send_keys(row['firstname'])

        # type the Lastname 
        keyword = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_uxSearchWideControl_txtLastName"]')
        keyword.send_keys(row['lastname'])

        # type the Title 
        keyword = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_uxSearchWideControl_txtKeyword"]')
        keyword.send_keys(row['title'])

        # Send information
        driver.find_element_by_xpath('//*[@id="lnkSearch"]').click()

        # Display restults 
        try:
            found = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_uxSearchLinks_Message"]')

        except: 
            pass

        try:        
            found = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_InlineTotalCount"]/div')
            found_1 = str(found.text)
            num_found = int(re.search(r'\d+', found_1).group(0))
            iterator = 3*num_found-1
            print(num_found)

        except:
            num_found = 0
            #pass

        print(found.text)
        time.sleep(3)

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
        time.sleep(2)


        try:      
            for i in range(1, iterator, 3):
                #time.sleep(3)
                try:
                    info_1 = driver.find_element_by_xpath('//*[@id="scrollpage1"]/div[%s]/div[2]'%(i))
                    information_1 = str(info_1.text)

                    real_name = str(row['name'])
                    university = str(row['university'])
                    univ_id = str(row['univ_id'])
                    person_id = str(row['person_id'])

                    data_1 = pd.DataFrame({ 'Univ_Id':[univ_id],'University':[university],'Person_Id':[person_id], \
                                           'Real_Name':[real_name],'Num_people':[num_found], 'Information_1':[information_1]})

                     # Get elements
                    data_1['Name_web'] = str(driver.find_element_by_xpath('//*[@id="scrollpage1"]/div[%s]/div[2]/div[1]/div[1]'%(i)).text)
                    # data_2['Date_Death'] = str(driver.find_element_by_xpath('//*[@id="scrollpage1"]/div[%s]/div[2]/div[1]/div[1]'%(i)).text)


                    #Fill up the text file with every person we found
                    with open("test_1.txt", "a") as myfile:
                        myfile.write( info_1.text + "\n")

                    #print(i)munich 
                    print(info_1.text)

                    # We append information extracted from data_1
                    scrap_data_1 = scrap_data_1.append(data_1)

                except:
                    pass


            for a in range(1, iterator, 3):
                time.sleep(3)

                data_2 = pd.DataFrame({ 'Univ_Id':[univ_id],'University':[university],'Person_Id':[person_id], \
                                           'Real_Name':[real_name],'Num_people':[num_found]})

                data_2['Name_web'] = str(driver.find_element_by_xpath('//*[@id="scrollpage1"]/div[%s]/div[2]/div[1]/div[1]'%(a)).text)


                driver.find_element_by_xpath('//*[@id="scrollpage1"]/div[%s]'%(a)).click()

                try :
                    time.sleep(3)
                    info_2 = driver.find_element_by_xpath('//*[@id="ctl00_MainContentPlaceholder_ObitTextContainer"]')
                    information_2 = str(info_2.text)            

                except :
                    info_2 = "THIS PERSON IS IN ANOTHER WEBPAGE"
                    information_2 = str(info_2)
                    print("THIS PERSON IS IN ANOTHER WEBPAGE : " + str(row['name']))

                finally :
                    data_2['Information_2'] = information_2

                    # Fill up the text file with every person we found
                    with open("test_2.txt", "a") as myfile:
                        myfile.write(information_2 + "\n")

                    # We append information extracted from data_2
                    scrap_data_2 = scrap_data_2.append(data_2)

                    print("BIOGRAPHY-" + information_2)
                    driver.back()

            driver.get('http://www.legacy.com/obituaries/legacy/obituary-search.aspx?daterange=30&countryid=0&stateid=all&affiliateid=all')        

        except:
            print("THIS PERSON HAS NOT BEEN FOUND : " + str(row['name']))
            # come back original webpage
            driver.get('http://www.legacy.com/obituaries/legacy/obituary-search.aspx?daterange=30&countryid=0&stateid=all&affiliateid=all')        

    except:
        driver.close()
        # Code to find people in genealogybank
        driver = webdriver.Chrome('C:/Users/quia/Desktop/chromedriver.exe')
        driver.get('http://www.legacy.com/obituaries/legacy/obituary-search.aspx?daterange=30&countryid=0&stateid=all&affiliateid=all')

        # Close advertisement
        driver.find_element_by_xpath('//*[@id="qcCmpButtons"]/button').click()

        # Maximize the window
        driver.maximize_window()

         # Set the state of last residence
        driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1_uxSearchWideControl_ddlCountry"]/option[11]').click()


        
        


Your search for "BELINDA AVALOS-COPELAN and Professor" did not find any obituaries in this newspaper.
THIS PERSON HAS NOT BEEN FOUND : AVALOS-COPELAN, BELINDA RENEE
Your search for "NEIL BAKER and Professor" did not find any obituaries in this newspaper.
THIS PERSON HAS NOT BEEN FOUND : BAKER, NEIL R
1
Displaying 1 out of 1 Results
Thomas Russell "Tom" Bean  
View Obituary
... and developed lifelong friendships with many of his professors and fellow students. While attending Rice University, he enlisted in the ROTC in order to serve his country. He taught civil engineering at the U.S. Army School of Engineering at Fort Belvoir, and later served in Vietnam as a captain in the Army Corps of Engineers. Tom was an avid supporter of the Kemper Contemporary Art museum, the Nelson-Atkins Art museum, the Kansas City Art Institute
Published in Kansas City Star on June 20, 2019
Get public records and additional details for Thomas Bean at PeopleFinders.com ›


C:\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


BIOGRAPHY-Thomas "Tom" Russell Bean Thomas "Tom" Russell Bean passed peacefully at his home on Saturday, June 1, 2019. Tom served as the City Architect of Kansas City, Missouri from 1980 to 2005, and represented the city in numerous projects that created a foundation for its continuing development. Some of the more notable projects he oversaw during his tenure were the renovation and expansion of Bartle Hall, the renovation of the Music Hall, the expansion of Kemper Arena, the revitalization of the City Market, and the construction of many police and fire stations. He also led the One-Percent-for-Art initiative, which provides funding for art in public buildings and spaces. As chairman of the Municipal Arts Commission, he oversaw 25 municipal public art installations, including the Sky Stations that top the pylons at Bartle Hall. Tom was an active member of the American Public Works Association and American Institute of Architects, including serving as President of the local AIA chapte

Listen to Obituary
7
Displaying 7 out of 7 Results
David A. Johnson (1952 - 2018)  
View Obituary
... resident. Dr. Johnson also completed fellowship training in Pediatric Ophthalmology at Children's National Medical Center in Washington, DC with Dr. Marshall Parks. Dr. Johnson was an Assistant Professor of Ophthalmology and Pediatrics at the Vanderbilt University School of Medicine and Director of Pediatric Ophthalmology at Vanderbilt Children's Hospital in the early 1990s. Following his academic career, Dr. Johnson became an accomplished private
Published in the Wilmington Star-News on Dec. 12, 2018
Get public records and additional details for David Johnson at PeopleFinders.com ›
Dr David Alfred Johnson (1938 - 2018)  
View Obituary
... grand child, and sister Ruth Sharon Johnson. Preceded in death by his parents. Graduate of Greenville College, LSU with his PhD in Chemistry. Dr. Johnson was a Professor of Chemistry at SAU for 38 years. He was a research Chemist for Argonne National

Listen to Obituary
BIOGRAPHY-David L. Johnson

David Lawrence Johnson was born to John Hugo Johnson and Jessie Evangeline Johnson March 16, 1923 in Moscow Idaho. He died in the arms of his family on January 19, 2014 in Seattle. He served in the U.S.
Army
Air Corp, and received his BS from the University of Idaho and his Ph.D. in Electrical Engineering from
Purdue University
. He joined the faculty of the Department of Electrical Engineering at the
as an Associate Professor in 1955. During his academic career, in addition to bringing the new fields of digital computing and computer science to the UW, he was a pioneer in artificial intelligence and computer engineering. He conducted seminal research into dolphin learning, took his family to Scotland where, as senior Fulbright Scholar, he worked with Jim Howe a leading AI researcher at the University of Edinburgh, and to Nantes France where he helped to develop the graduate program in engineering at l'Universite; de Nantes. Complementing 

...

THIS PERSON HAS NOT BEEN FOUND : KENNEL, JULIE ANNE
Your search for "JASON KENTNER and Professor" did not find any obituaries in this newspaper.
THIS PERSON HAS NOT BEEN FOUND : KENTNER, JASON
Your search for "JUSTIN KIEFFER and Professor" did not find any obituaries in this newspaper.
THIS PERSON HAS NOT BEEN FOUND : KIEFFER, JUSTIN DAVID
Your search for "MENACHEM KIMCHI and Professor" did not find any obituaries in this newspaper.
THIS PERSON HAS NOT BEEN FOUND : KIMCHI, MENACHEM
Your search for "LONNIE KING and Professor" did not find any obituaries in this newspaper.
THIS PERSON HAS NOT BEEN FOUND : KING, LONNIE JAMES
Your search for "VALERIE KINLOCH and Professor" did not find any obituaries in this newspaper.
THIS PERSON HAS NOT BEEN FOUND : KINLOCH, VALERIE F
Your search for "CHARLOTTE KIRSCHNER and Professor" did not find any obituaries in this newspaper.
THIS PERSON HAS NOT BEEN FOUND : KIRSCHNER, CHARLOTTE LIN
Your search for "BOBBIE KITE and Professor" did not find any obitu

Print
BIOGRAPHY-David A. Miller, MDCorpus Christi, TXDavid Arthur Miller, MD, 68, passed away peacefully on May 21, 2015, after a courageous 12 year battle with cancer. He was born on July 6, 1946, to Paul and Mary Miller in Canton, Ohio, where he was raised. David graduated summa cum laude with a degree in Biology from Bowling Green State University in 1968. He then attended the
School of Medicine and graduated cum laude with a degree in Medicine in 1972. Dr. Miller was board certified in Internal and Pulmonary Medicine and a Fellow in the American College of Chest Physicians. Dr. Miller moved his family to Corpus Christi, Texas, in 1977 where he established a successful medical practice, specializing in pulmonary medicine. He will always be remembered for the love, care and empathy he had for his patients. In the early 1980's, Dr. Miller was one of the first doctors in South Texas to actively care for AIDS patients. His first AIDS patient referral came from Dr. Hector P. Garcia Jr. H

Get public records and additional details for PETER ROGERS at PeopleFinders.com ›
PETER ROGERS  
View Obituary
... and then received a degree in Engineering from the University of Liverpool. Subsequently he came to the United States for a master's degree at Northwestern University, followed by a PhD at Harvard University. He served his entire career as the Gordon McKay Professor of Environmental Engineering at Harvard, with his primary appointment in the School of Engineering and Applied Sciences. He was survived by his wife Suzanne, his two sons Christopher and Justin, his
Published in The Boston Globe on Feb. 18, 2018
Get public records and additional details for PETER ROGERS at PeopleFinders.com ›
THIS PERSON IS IN ANOTHER WEBPAGE : ROGERS, PETER F
BIOGRAPHY-THIS PERSON IS IN ANOTHER WEBPAGE
BIOGRAPHY-Print
Your search for "YOLANDA RORY and Professor" did not find any obituaries in this newspaper.
THIS PERSON HAS NOT BEEN FOUND : RORY, YOLANDA
Your search for "NATHAN ROSENBERG and P

Get public records and additional details for GEORGE SMITHY at PeopleFinders.com ›
BIOGRAPHY-Print
BIOGRAPHY-Print
THIS PERSON IS IN ANOTHER WEBPAGE : SMITH, GEORGE
BIOGRAPHY-THIS PERSON IS IN ANOTHER WEBPAGE
BIOGRAPHY-Print
BIOGRAPHY-Print
Your search for "JOHN SNYDER and Professor" did not find any obituaries in this newspaper.
THIS PERSON HAS NOT BEEN FOUND : SNYDER, JOHN ROBERT
Your search for "AMY SONNTAG and Professor" did not find any obituaries in this newspaper.
THIS PERSON HAS NOT BEEN FOUND : SONNTAG, AMY
Your search for "DAWN STARR and Professor" did not find any obituaries in this newspaper.
THIS PERSON HAS NOT BEEN FOUND : STARR, DAWN
Your search for "KURT STEVENSON and Professor" did not find any obituaries in this newspaper.
THIS PERSON HAS NOT BEEN FOUND : STEVENSON, KURT B
Your search for "LATONIA STINER-JONES and Professor" did not find any obituaries in this newspaper.
THIS PERSON HAS NOT BEEN FOUND : STINER-JONES, LATONIA M
Your search for "TERRELL STRAYHORN and Pr

Print
BIOGRAPHY-Print
BIOGRAPHY-Print
THIS PERSON IS IN ANOTHER WEBPAGE : WILLIAMS, DAVID B
BIOGRAPHY-THIS PERSON IS IN ANOTHER WEBPAGE
BIOGRAPHY-Print
THIS PERSON IS IN ANOTHER WEBPAGE : WILLIAMS, DAVID B
BIOGRAPHY-THIS PERSON IS IN ANOTHER WEBPAGE
BIOGRAPHY-David T. Williams

91, of Indianapolis, who passed away on February 25, 2016 was born on November 21, 1924 to the late: George & Pearl Williams, 2 Salvation Army Officers. Dad was the oldest of 6 children. Dad lived by faith his entire life . By faith he served in
in the Air Core. By faith he entered a music conservatory in N.Y.C. By faith he became a Fulbright Scholar studying at the Mozarteaum. By faith he sang: in the states, on Broadway and at Opera Houses in Europe. He sang with the love of his life, Phyllis Wilcox Williams. (Dad was especially proud of being in the early live TV performance of Billy Bud on NBC OPERA, 1952. It can be seen on YouTube.) By faith dad became: Warrior, Husband, Singer, Father, Actor, Director, Pro

In [672]:
scrap_data_1

,Date_Death,Information_1,Name_web,Num_people,Person_Id,Real_Name,Univ_Id,University
0,NaN,"Thomas Russell ""Tom"" Bean \nView Obituary\n.....","Thomas Russell ""Tom"" Bean",1.0,132898,"BEAN, THOMAS L",156,"Ohio State University, The"
0,NaN,"Dr. William August ""Bill"" Jensen Ph.D. (1927 -...","Dr. William August ""Bill"" Jensen Ph.D. (1927 -...",1.0,175869,"JENSEN, WILLIAM A",156,"Ohio State University, The"
0,NaN,David A. Johnson (1952 - 2018) \nView Obituar...,David A. Johnson (1952 - 2018),7.0,301054,"JOHNSON, DAVID L",156,"Ohio State University, The"
0,NaN,Dr David Alfred Johnson (1938 - 2018) \nView ...,Dr David Alfred Johnson (1938 - 2018),7.0,301054,"JOHNSON, DAVID L",156,"Ohio State University, The"
0,NaN,David L. Johnson (1923 - 2014) \nView Obituar...,David L. Johnson (1923 - 2014),7.0,301054,"JOHNSON, DAVID L",156,"Ohio State University, The"
0,NaN,David Mark Johnson \nView Obituary\n... JOHNS...,David Mark Johnson,7.0,301054,"JOHNSON, DAVID L",156,"Ohio State University, The"
0,NaN,David Johnson (1944 - 2017) \nView Obituary\n...,David Johnson (1944 - 2017),7.0,301054,"JOHNSON, DAVID L",156,"Ohio State University, The"
0,NaN,David Johnson* (1941 - 2014) \nView Obituary\...,David Johnson* (1941 - 2014),7.0,301054,"JOHNSON, DAVID L",156,"Ohio State University, The"
0,NaN,David Johnson (1944 - 2017) \nView Obituary\n...,David Johnson (1944 - 2017),7.0,301054,"JOHNSON, DAVID L",156,"Ohio State University, The"
0,NaN,Leroy A. (Lee) Blaske Ph.D.* (1923 - 2014) \n...,Leroy A. (Lee) Blaske Ph.D.* (1923 - 2014),6.0,152502,"LEE, L. JAMES",156,"Ohio State University, The"


In [673]:
scrap_data_2

,Date_Death,Information_2,Name_web,Num_people,Person_Id,Real_Name,Univ_Id,University
0,NaN,"Thomas ""Tom"" Russell Bean Thomas ""Tom"" Russell...","Thomas Russell ""Tom"" Bean",1.0,132898,"BEAN, THOMAS L",156,"Ohio State University, The"
0,NaN,"Jensen Dr. William August ""Bill"" Jensen, Ph.D....","Dr. William August ""Bill"" Jensen Ph.D. (1927 -...",1.0,175869,"JENSEN, WILLIAM A",156,"Ohio State University, The"
0,NaN,"DAVID A. JOHNSON, MD,PHD David A. Johnson, MD,...",David A. Johnson (1952 - 2018),7.0,301054,"JOHNSON, DAVID L",156,"Ohio State University, The"
0,NaN,"Johnson, Dr David Alfred Of Spring Arbor, MI p...",Dr David Alfred Johnson (1938 - 2018),7.0,301054,"JOHNSON, DAVID L",156,"Ohio State University, The"
0,NaN,David L. Johnson\n\nDavid Lawrence Johnson was...,David L. Johnson (1923 - 2014),7.0,301054,"JOHNSON, DAVID L",156,"Ohio State University, The"
0,NaN,"JOHNSON, DAVID MARK\nDavid Mark Johnson died a...",David Mark Johnson,7.0,301054,"JOHNSON, DAVID L",156,"Ohio State University, The"
0,NaN,"Leroy A. (Lee) Blaske, PhD\n\nOctober 26, 1923...",Leroy A. (Lee) Blaske Ph.D.* (1923 - 2014),6.0,152502,"LEE, L. JAMES",156,"Ohio State University, The"
0,NaN,Leland (Lee) Leonard Burger\n\nLeland L. Burge...,Leland Leonard (Lee) Burger (1917 - 2016),6.0,152502,"LEE, L. JAMES",156,"Ohio State University, The"
0,NaN,THIS PERSON IS IN ANOTHER WEBPAGE,Lawrence Lynn Lee (1924 - 2017),6.0,152502,"LEE, L. JAMES",156,"Ohio State University, The"
0,NaN,THIS PERSON IS IN ANOTHER WEBPAGE,Lawrence Lynn Lee (1924 - 2017),6.0,152502,"LEE, L. JAMES",156,"Ohio State University, The"


In [674]:
# This lines eliminate new line´s creation "\n"

scrap_data_1["Information_1"] = scrap_data_1["Information_1"].str.replace('\n',' ', regex=True) 
scrap_data_2["Information_2"] = scrap_data_2["Information_2"].str.replace('\n',' ', regex=True) 
# scrap_data_2.iloc[50,1]#.replace("\n"," ")

In [675]:
# Directory to save the file 
scrap_data_1.to_csv('C:/Users/quia/Desktop/Faculty_Rosters/scrap_data_1_ohio_state.csv', encoding='utf-8', sep=',')

In [676]:
# Directory to save the file 
scrap_data_2.to_csv('C:/Users/quia/Desktop/Faculty_Rosters/scrap_data_2_ohio_state.csv', encoding='utf-8',sep=',')